In [80]:
import boto3
import sagemaker
import pandas as pd

sess   = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name='sagemaker', region_name=region)

# Specify the S3 Location of the Features

In [81]:
%store -r scikit_processing_job_s3_output_prefix

In [82]:
print('Previous Scikit Processing Job Name: {}'.format(scikit_processing_job_s3_output_prefix))

Previous Scikit Processing Job Name: sagemaker-scikit-learn-2020-03-30-03-34-18-188


In [83]:
prefix_train = '{}/output/bert-train'.format(scikit_processing_job_s3_output_prefix)
prefix_validation = '{}/output/bert-validation'.format(scikit_processing_job_s3_output_prefix)
prefix_test = '{}/output/bert-test'.format(scikit_processing_job_s3_output_prefix)

path_train = './{}'.format(prefix_train)
path_validation = './{}'.format(prefix_validation)
path_test = './{}'.format(prefix_test)

train_s3_uri = 's3://{}/{}'.format(bucket, prefix_train)
validation_s3_uri = 's3://{}/{}'.format(bucket, prefix_validation)
test_s3_uri = 's3://{}/{}'.format(bucket, prefix_test)

In [89]:
s3_input_train_data = sagemaker.s3_input(s3_data=train_s3_uri) #, content_type='text/csv')
s3_input_validation_data = sagemaker.s3_input(s3_data=validation_s3_uri) #, content_type='text/csv')
s3_input_test_data = sagemaker.s3_input(s3_data=test_s3_uri) #, content_type='text/csv')

print(s3_input_train_data.config)
print(s3_input_validation_data.config)
print(s3_input_test_data.config)

{'DataSource': {'S3DataSource': {'S3DataType': 'S3Prefix', 'S3Uri': 's3://sagemaker-us-east-1-835319576252/sagemaker-scikit-learn-2020-03-30-03-34-18-188/output/bert-train', 'S3DataDistributionType': 'FullyReplicated'}}}
{'DataSource': {'S3DataSource': {'S3DataType': 'S3Prefix', 'S3Uri': 's3://sagemaker-us-east-1-835319576252/sagemaker-scikit-learn-2020-03-30-03-34-18-188/output/bert-validation', 'S3DataDistributionType': 'FullyReplicated'}}}
{'DataSource': {'S3DataSource': {'S3DataType': 'S3Prefix', 'S3Uri': 's3://sagemaker-us-east-1-835319576252/sagemaker-scikit-learn-2020-03-30-03-34-18-188/output/bert-test', 'S3DataDistributionType': 'FullyReplicated'}}}


In [90]:
!cat src_bert_tf2/tf_bert_reviews.py

import sys
import subprocess
import argparse
import json
#subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'tensorflow-gpu==2.2.0-rc2'])
subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'bert-for-tf2'])
subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'sentencepiece'])

import tensorflow as tf
print(tf.__version__)

import boto3
import pandas as pd

import os
import math
import datetime

from tqdm import tqdm

import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow import keras
from glob import glob 

from bert.model import BertModelLayer
from bert.loader import StockBertConfig, map_stock_config_to_params, load_stock_weights
from bert.tokenization.bert_tokenization import FullTokenizer

from sklearn.metrics import confusion_matrix, classification_report

#train = pd.read_csv('./data/amazon_reviews_us_Digital_Software_v1_00.tsv.gz', delimiter='\t')[:100]
#test = pd.read_csv('./data/amazon_reviews_us_Digital_Software_v1_00

In [91]:
from sagemaker.tensorflow import TensorFlow

model_output_path = 's3://{}/models/tf2-bert'.format(bucket)

bert_estimator = TensorFlow(entry_point='tf_bert_reviews.py',
                         source_dir='src_bert_tf2',
                         role=role,
                         train_instance_count=1, # 1 is actually faster due to communication overhead with >1
                         train_instance_type='ml.p3.16xlarge',
                         py_version='py3',
                         framework_version='2.0.0',
                         output_path=model_output_path,
#                         hyperparameters={'model_type':'bert',
#                                          'model_name': 'bert-base-cased'},
                         enable_cloudwatch_metrics=True)

# Train the model

In [92]:
bert_estimator.fit(inputs={'train': s3_input_train_data, 
                           'validation': s3_input_validation_data,}, 
                   wait=False)

In [93]:
training_job_name = bert_estimator.latest_training_job.name
print('training_job_name:  {}'.format(training_job_name))

training_job_name:  tensorflow-training-2020-03-31-05-58-21-428


In [94]:
# from sagemaker.tensorflow import TensorFlow

# bert_estimator = TensorFlow.attach(training_job_name=training_job_name)

In [95]:
from IPython.core.display import display, HTML

display(HTML('<b>Review <a href="https://console.aws.amazon.com/sagemaker/home?region={}#/jobs/{}">Training Job</a> After About 5 Minutes</b>'.format(region, training_job_name)))


In [96]:
from IPython.core.display import display, HTML

display(HTML('<b>Review <a href="https://console.aws.amazon.com/cloudwatch/home?region={}#logStream:group=/aws/sagemaker/TrainingJobs;prefix={};streamFilter=typeLogStreamPrefix">CloudWatch Logs</a> After About 5 Minutes</b>'.format(region, training_job_name)))


In [97]:
from IPython.core.display import display, HTML

training_job_s3_output_prefix = 'models/tf2-bert/{}'.format(training_job_name) # 'models/tf-bert/script-mode/training-runs/{}'.format(training_job_name)

display(HTML('<b>Review <a href="https://s3.console.aws.amazon.com/s3/buckets/{}/{}/?region={}&tab=overview">S3 Output Data</a> After The Training Job Has Completed</b>'.format(bucket, training_job_s3_output_prefix, region)))


# Download and Load the Trained Model

In [ ]:
# download the model artifact from AWS S3

!aws s3 cp $model_output_path/$training_job_name/output/model.tar.gz ./models/tf2-bert/

#!aws s3 cp s3://sagemaker-us-east-1-835319576252/models/tf-bert/script-mode/training-runs/tensorflow-training-2020-03-24-04-41-39-405/output/model.tar.gz ./models/tf2-bert/

In [ ]:
import tarfile
import pickle as pkl

tar = tarfile.open('./models/tf2-bert/model.tar.gz')
tar.extractall(path='./models/tf2-bert')
tar.close()

In [ ]:
!ls -al ./models/tf2-bert

In [ ]:
!ls -al ./models/tf2-bert/0

In [ ]:
# Must upgrade wrapt before installing TF
!pip install -q wrapt --upgrade --ignore-installed
!pip install -q tensorflow==1.15.2
!pip install -q tensorflow-hub==0.7.0
!pip install -q bert-tensorflow==1.0.1

# Load the model

In [ ]:
saved_model = tf.saved_model.load_v2(
    './models/tf2-bert/0',
    tags=None
)

In [ ]:
# TODO:  Load the Keras model (or weights?) using the most appopriate mechanism (not necessarily this way) 
#loaded_model = create_model(data.max_seq_len, adapter_size=None)
#loaded_model.load_weights("movie_reviews.h5")


In [ ]:
_, train_acc = loaded_model.evaluate(data.train_x, data.train_y)
_, test_acc = loaded_model.evaluate(data.test_x, data.test_y)

print("train acc", train_acc)
print(" test acc", test_acc)

In [ ]:
inference = saved_model.signatures["serving_default"]
print(inference.inputs)
print(inference.structured_outputs)

# Predict 


In [ ]:
from bert import tokenization
import tensorflow_hub as hub

BERT_MODEL_HUB = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"

def create_tokenizer_from_hub_module():
    """Get the vocab file and casing info from the Hub module."""
    with tf.Graph().as_default():
        bert_module = hub.Module(BERT_MODEL_HUB)
        tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
        with tf.Session() as sess:
            vocab_file, do_lower_case = sess.run([tokenization_info["vocab_file"],
                                                tokenization_info["do_lower_case"]])
      
        return tokenization.FullTokenizer(vocab_file=vocab_file,
                                               do_lower_case=do_lower_case)
   

In [ ]:
def get_predict_features(features, seq_length):
  all_input_ids = []
  all_input_mask = []
  all_segment_ids = []
  all_label_ids = []

  for feature in features:
    all_input_ids.append(feature.input_ids)
    all_input_mask.append(feature.input_mask)
    all_segment_ids.append(feature.segment_ids)
    all_label_ids.append(feature.label_id)

    batch_size = 32

  num_examples = len(features)

  # This is for demo purposes and does NOT scale to large data sets. We do
  # not use Dataset.from_generator() because that uses tf.py_func which is
  # not TPU compatible. The right way to load data is with TFRecordReader.
#   d = tf.data.Dataset.from_tensor_slices({
#     "input_ids":
#         tf.constant(
#             all_input_ids, shape=[num_examples, seq_length],
#             dtype=tf.int32),
#     "input_mask":
#         tf.constant(
#             all_input_mask,
#             shape=[num_examples, seq_length],
#             dtype=tf.int32),
#     "segment_ids":
#         tf.constant(
#             all_segment_ids,
#             shape=[num_examples, seq_length],
#             dtype=tf.int32),
#     "label_ids":
#         tf.constant(all_label_ids, shape=[num_examples], dtype=tf.int32),
#   })

#   d = d.batch(batch_size=batch_size, drop_remainder=False)

#  return d

  input_ids = tf.constant(
             all_input_ids, shape=[num_examples, seq_length],
             dtype=tf.int32)

  input_mask = tf.constant(
             all_input_mask,
             shape=[num_examples, seq_length],
             dtype=tf.int32)

  segment_ids = tf.constant(
             all_segment_ids,
             shape=[num_examples, seq_length],
             dtype=tf.int32)

  label_ids = tf.constant(all_label_ids, shape=[num_examples], dtype=tf.int32),

  return input_ids, input_mask, segment_ids, label_ids

In [ ]:
from src_bert_tf import amazon_run_classifier

MAX_SEQ_LENGTH = 128
LABEL_VALUES = [1, 2, 3, 4, 5]

def predict(in_sentences):
    tokenizer = create_tokenizer_from_hub_module()
    print('**** TOKENIZER {}****'.format(tokenizer))
    
    input_examples = [amazon_run_classifier.InputExample(guid="", text_a = x, text_b = None, label = -1) for x in in_sentences]
    input_features = amazon_run_classifier.convert_examples_to_features(input_examples, LABEL_VALUES, MAX_SEQ_LENGTH, tokenizer)

#    predict_input_fn = amazon_run_classifier.input_fn_builder(features=input_features, seq_length=MAX_SEQ_LENGTH, is_training=False, drop_remainder=False)
#    predictions = estimator.predict(predict_input_fn)
    input_ids, input_mask, segment_ids, label_ids = get_predict_features(input_features, MAX_SEQ_LENGTH)
    print(type(input_ids))

    inference(input_ids=input_ids, input_mask=input_mask, segment_ids=segment_ids, label_ids=label_ids)

    return [(sentence, prediction['probabilities'], LABEL_VALUES[prediction['labels']]) for sentence, prediction in zip(in_sentences, predictions)]


In [ ]:
import numpy as np

pred_sentences = [
  "That movie was absolutely awful",
  "The acting was a bit lacking",
  "The film was creative and surprising",
  "Absolutely fantastic!"
]

np_list = np.asarray(pred_sentences)
tensor_list = tf.convert_to_tensor(np_list)
predictions = predict(tensor_list)